In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rohlik-orders-forecasting-challenge/test_calendar.csv
/kaggle/input/rohlik-orders-forecasting-challenge/solution_example.csv
/kaggle/input/rohlik-orders-forecasting-challenge/train.csv
/kaggle/input/rohlik-orders-forecasting-challenge/test.csv
/kaggle/input/rohlik-orders-forecasting-challenge/train_calendar.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

In [3]:
train_df = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/train.csv')
test_df = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/test.csv')
train_calendar_df = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/train_calendar.csv')
test_calendar_df = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/test_calendar.csv')

In [4]:
train_df.head()

,warehouse,date,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,user_activity_1,user_activity_2,id
0,Prague_1,2020-12-05,6895.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1722.0,32575.0,Prague_1_2020-12-05
1,Prague_1,2020-12-06,6584.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1688.0,32507.0,Prague_1_2020-12-06
2,Prague_1,2020-12-07,7030.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1696.0,32552.0,Prague_1_2020-12-07
3,Prague_1,2020-12-08,6550.0,NaN,0,0,0,0,0,0,0,0.0,0,0.8,0.0,1681.0,32423.0,Prague_1_2020-12-08
4,Prague_1,2020-12-09,6910.0,NaN,0,0,0,0,0,0,0,0.0,0,0.5,0.0,1704.0,32410.0,Prague_1_2020-12-09


In [5]:
test_df.head()

,warehouse,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id
0,Prague_1,2024-03-16,NaN,0,0,0,0,Prague_1_2024-03-16
1,Prague_1,2024-03-17,NaN,0,0,0,0,Prague_1_2024-03-17
2,Prague_1,2024-03-18,NaN,0,0,0,0,Prague_1_2024-03-18
3,Prague_1,2024-03-19,NaN,0,0,0,0,Prague_1_2024-03-19
4,Prague_1,2024-03-20,NaN,0,0,0,0,Prague_1_2024-03-20


In [6]:
train_calendar_df.head()

,date,holiday_name,holiday,shutdown,mini_shutdown,warehouse_limited,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,warehouse
0,2022-03-09,NaN,0,0,0,0,0,1,0,0,0.0,0,0.0,0.0,Prague_1
1,2019-07-03,NaN,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,Prague_1
2,2020-05-18,NaN,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,Prague_1
3,2021-11-09,NaN,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,Prague_1
4,2023-08-02,NaN,0,0,0,0,0,0,0,0,0.0,0,2.5,0.0,Prague_1


In [7]:
test_calendar_df.head()

,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,warehouse
0,2024-11-17,Den boje za svobodu a demokracii,1,0,0,0,Prague_1
1,2024-07-04,NaN,0,0,0,0,Prague_1
2,2024-08-04,NaN,0,0,0,0,Prague_1
3,2024-12-01,NaN,0,0,0,0,Prague_1
4,2024-07-24,NaN,0,0,0,0,Prague_1


In [8]:
# Convert the 'date' column to datetime format
train_df['date'] = pd.to_datetime(train_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])

# Convert the 'date' columns in the calendar dataframes to datetime format
train_calendar_df['date'] = pd.to_datetime(train_calendar_df['date'])
test_calendar_df['date'] = pd.to_datetime(test_calendar_df['date'])

# Extracting date features
train_df['day_of_week'] = train_df['date'].dt.dayofweek
train_df['month'] = train_df['date'].dt.month
train_df['is_weekend'] = train_df['day_of_week'].isin([5, 6]).astype(int)

test_df['day_of_week'] = test_df['date'].dt.dayofweek
test_df['month'] = test_df['date'].dt.month
test_df['is_weekend'] = test_df['day_of_week'].isin([5, 6]).astype(int)

# Merging train and test datasets with their respective calendars
train_merged = pd.merge(train_df, train_calendar_df, on=['date', 'warehouse'], how='left')
test_merged = pd.merge(test_df, test_calendar_df, on=['date', 'warehouse'], how='left')

print("Merged Train Dataset:")
print(train_merged.head())

print("\nMerged Test Dataset:")
print(test_merged.head())

Merged Train Dataset:
  warehouse       date  orders holiday_name_x  holiday_x  shutdown_x  \
0  Prague_1 2020-12-05  6895.0            NaN          0           0   
1  Prague_1 2020-12-06  6584.0            NaN          0           0   
2  Prague_1 2020-12-07  7030.0            NaN          0           0   
3  Prague_1 2020-12-08  6550.0            NaN          0           0   
4  Prague_1 2020-12-09  6910.0            NaN          0           0   

   mini_shutdown_x  shops_closed_x  winter_school_holidays_x  \
0                0               0                         0   
1                0               0                         0   
2                0               0                         0   
3                0               0                         0   
4                0               0                         0   

   school_holidays_x  ...  mini_shutdown_y  warehouse_limited  shops_closed_y  \
0                  0  ...                0                  0               0  

In [9]:
train_merged.columns

Index(['warehouse', 'date', 'orders', 'holiday_name_x', 'holiday_x',
       'shutdown_x', 'mini_shutdown_x', 'shops_closed_x',
       'winter_school_holidays_x', 'school_holidays_x', 'blackout_x',
       'mov_change_x', 'frankfurt_shutdown_x', 'precipitation_x', 'snow_x',
       'user_activity_1', 'user_activity_2', 'id', 'day_of_week', 'month',
       'is_weekend', 'holiday_name_y', 'holiday_y', 'shutdown_y',
       'mini_shutdown_y', 'warehouse_limited', 'shops_closed_y',
       'winter_school_holidays_y', 'school_holidays_y', 'blackout_y',
       'mov_change_y', 'frankfurt_shutdown_y', 'precipitation_y', 'snow_y'],
      dtype='object')

In [10]:
test_merged.columns

Index(['warehouse', 'date', 'holiday_name_x', 'holiday_x', 'shops_closed_x',
       'winter_school_holidays_x', 'school_holidays_x', 'id', 'day_of_week',
       'month', 'is_weekend', 'holiday_name_y', 'holiday_y', 'shops_closed_y',
       'winter_school_holidays_y', 'school_holidays_y'],
      dtype='object')

In [11]:
# list of columns to drop
cols_to_drop = ['shutdown_x', 'mini_shutdown_x', 'blackout_x', 'mov_change_x', 
                            'frankfurt_shutdown_x', 'precipitation_x', 'snow_x', 
                            'user_activity_1', 'user_activity_2']

# Drop the specified columns
train_final = train_merged.drop(columns=cols_to_drop)

# these columns don't exist in the test dataset, so no need to drop them from test_final
test_final = test_merged

print("Final Train Dataset:")
print(train_final.head())

print("\nFinal Test Dataset:")
print(test_final.head())

Final Train Dataset:
  warehouse       date  orders holiday_name_x  holiday_x  shops_closed_x  \
0  Prague_1 2020-12-05  6895.0            NaN          0               0   
1  Prague_1 2020-12-06  6584.0            NaN          0               0   
2  Prague_1 2020-12-07  7030.0            NaN          0               0   
3  Prague_1 2020-12-08  6550.0            NaN          0               0   
4  Prague_1 2020-12-09  6910.0            NaN          0               0   

   winter_school_holidays_x  school_holidays_x                   id  \
0                         0                  0  Prague_1_2020-12-05   
1                         0                  0  Prague_1_2020-12-06   
2                         0                  0  Prague_1_2020-12-07   
3                         0                  0  Prague_1_2020-12-08   
4                         0                  0  Prague_1_2020-12-09   

   day_of_week  ...  mini_shutdown_y  warehouse_limited shops_closed_y  \
0            5  ...  

In [12]:
# Creating a 7-day lag feature
train_final['orders_lag_7'] = train_final.groupby('warehouse')['orders'].shift(7)

# Creating a 7-day rolling average feature
train_final['orders_rolling_7'] = train_final.groupby('warehouse')['orders'].rolling(7).mean().shift(1).reset_index(0, drop=True)

# Drop any rows with NaN values
train_final = train_final.dropna()

print("Train Dataset with Lag and Rolling Features:")
print(train_final.head())


Train Dataset with Lag and Rolling Features:
    warehouse       date  orders            holiday_name_x  holiday_x  \
19   Prague_1 2020-12-24  4327.0             Christmas Eve          1   
20   Prague_1 2020-12-26  7784.0         2nd Christmas Day          1   
26   Prague_1 2021-01-01  3754.0             New Years Day          1   
92   Prague_1 2021-03-08  8212.0  International womens day          0   
117  Prague_1 2021-04-02  9224.0               Good Friday          1   

     shops_closed_x  winter_school_holidays_x  school_holidays_x  \
19                0                         0                  0   
20                1                         0                  0   
26                1                         0                  0   
92                0                         1                  0   
117               0                         0                  0   

                      id  day_of_week  ...  shops_closed_y  \
19   Prague_1_2020-12-24            3  ...   

# Model 

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error,r2_score
from sklearn.preprocessing import OneHotEncoder

In [19]:
# Identify categorical features
categorical_features = ['warehouse', 'holiday_name_x', 'holiday_name_y']

# Apply one hot encoding to the training data
X = pd.get_dummies(train_final.drop(columns=['orders', 'id', 'date']), columns=categorical_features)

# Define the target variable
y = train_final['orders']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes of the splits
print(f"Training data shape: {X_train.shape}, Validation data shape: {X_val.shape}")

# Initialize the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = rf_model.predict(X_val)

# Evaluate the model
mape = mean_absolute_percentage_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
r2 = r2_score(y_val, y_pred)

print(f"Validation MAPE: {mape:.4f}")
print(f"Validation RMSE: {rmse:.4f}")
print(f"Validation R^2 Score: {r2:.4f}")

Training data shape: (166, 76), Validation data shape: (42, 76)
Validation MAPE: 0.1000
Validation RMSE: 586.9758
Validation R^2 Score: 0.9110


In [20]:
# Prepare the test data
X_test = test_final.drop(columns=['id', 'date'])

# Apply one hot encoding to the test data with the same columns as in the training data
X_test = pd.get_dummies(X_test, columns=categorical_features)

# Align the test set's columns with the training set
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Make predictions
y_test_pred = rf_model.predict(X_test)

In [21]:
# Prepare the submission Data
submission = pd.DataFrame({
    'ID': test_final['id'],  # Ensure id matches the format in the test data
    'ORDERS': y_test_pred
})

# Save the file
submission.to_csv('submission.csv', index=False)
print("Submission file created: 'submission.csv'")

Submission file created: 'submission.csv'


In [23]:
# Display the Data
submission

,ID,ORDERS
0,Prague_1_2024-03-16,1640.56
1,Prague_1_2024-03-17,1649.82
2,Prague_1_2024-03-18,1578.46
3,Prague_1_2024-03-19,1578.46
4,Prague_1_2024-03-20,1545.23
...,...,...
392,Budapest_1_2024-05-11,1640.56
393,Budapest_1_2024-05-12,1649.82
394,Budapest_1_2024-05-13,1578.46
395,Budapest_1_2024-05-14,1578.46
